<a href="https://colab.research.google.com/github/nibst/T2-AI/blob/main/T2_2023_02_An%C3%A1lise_de_agrupamentos_com_K_means.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Instituto de Informática - UFRGS**
## Disciplina INF01017 - Aprendizado de Máquina
#### *Prof. Anderson Rocha Tavares*
#### Créditos pelo Material: *Profa. Mariana Recamonde-Mendoza*
### **Trabalho 2 - Análise de agrupamentos com K-means**
<br>

**Grupo**:
*   (nome e cartão UFRGS)
*   (nome e cartão UFRGS)
*   (nome e cartão UFRGS)


---
***Observação:*** Este notebook é disponibilizado aos alunos como ponto de partida para o desenvolvimento do trabalho prático 2 (T2) da disciplina INF01017.


---

***ENTREGA:*** A entrega deste trabalho deve ser feita enviando o **link do Google Colab** com a solução do grupo, com a opção de deixar **as saídas do notebook salvas**, e o .ipynb exportado do Google Colab, contendo todas as saidas.



# Segmentação de Clientes com Algoritmo K-Means

O conjunto de dados a ser utilizado nesse trabalho foi adaptado de um problema de classificação de risco de crédito para clientes bancários. Os dados a serem analisados não possuem rótulos (classes) e possuem apenas um subconjunto previamente selecionado dos atributos utilizados para descrever as instâncias analisadas. O objetivo é realizar o processo de segmentação de clientes que solicitaram crédito bancário, que consiste em separar os clientes em grupos menores com base em características comuns entre eles. A partir destes grupos gerados (aqui denominados clusters), a empresa pode oferecer uma comunicação mais assertiva e personalizada aos seus clientes, e melhor compreender sobre os interesses e necessidades dos seus clientes ao traçar perfis de clientes (também denominados *personas*).


Os atributos disponíveis estão descritos abaixo:

*   **Age.** Idade (numérico)
*   **Credit amount.** Valor do crédito (numérico, em DM - Deutsch Mark)
*   **Duration.** Duração do empréstimo (numérico, em meses)
*   **Sex.** Sexo (categórico: masculino, feminino)
*   **Job.** Emprego (categórico: 0 - não qualificado e não residente, 1 - não qualificado e residente, 2 - qualificado, 3 - altamente qualificado)
*   **Housing.** Imóvel (categórico: próprio, alugado ou gratuito)
*   **Saving Account.** Poupança (categórico: pequena, moderada, bastante rico, rico)
*   **Checking Account.** Conta corrente (categórico: pequena, moderada, bastante rico, rico)
*   **Purpose.** Finalidade (categórico: carro, móveis/equipamentos, rádio/TV, eletrodomésticos, reparos, educação, negócios, férias/outros)





In [ ]:
##importando bilbiotecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")

Lendo os dados e visualizando a estrutura para as primeiras instâncias

In [ ]:
data = pd.read_csv("https://drive.google.com/uc?export=view&id=1Jp-y1djRI3sCT6_JTBkfLn1FOL4qMHsv",  )

In [ ]:
data.head()

A primeira coluna pode ser removida, pois é apenas um identificador (da instância ou da linha).

In [ ]:
data.drop(data.columns[0], inplace=True, axis=1)

Inspecionando o tamanho do base de dados, os tipos dos atributos e a ocorrência de valores faltantes.

In [ ]:
print("O conjunto de dados possui {} instâncias (clientes) e {} colunas (atributos).".format(data.shape[0],data.shape[1]))

In [ ]:
print("Valores faltantes (%) por atributo:\n{}".format((data.isnull().sum()/data.shape[0])*100))

Os valores faltantes ocorrem nos atributos Saving accounts e Checking account. Provavelmente são clientes que não possuem algum destes tipos de conta. Neste caso, optamos por não imputar os valores faltantes.

In [ ]:
print("Tipo de dado por atributo:\n{}".format(data.dtypes))

Embora o atributo Job esteja codificado como inteiro, ele possui uma interpretação **categórica**. Vamos fazer a conversão de tipo e separar os atributos em vetores de categóricos e numéricos para facilitar a análise exploratória dos dados.

In [ ]:
## Converte o atributo job para object.
data_types_dict = {'Job': object}

data = data.astype(data_types_dict)

## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(data.select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(data.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

## Separa os dados em dois dataframes, de atributos numéricos e categóricos
data_num = data[num_columns]
data_cat = data[cat_columns]

In [ ]:
data_num.describe()

A análise de agrupamento para identificar perfis de clientes será feita a partir dos dados numéricos: **Age, Credit Amount, Duration**.


Uma vez definidos os clusters, os demais atributos serão empregados para uma interpretação mais aprofundada destes clusters e dos respectivos perfis de clientes que representam.




### **Análise Exploratória dos Dados**

Nas células abaixo, vamos realizar uma análise exploratória dos dados.

Primeiramente, vamos observar a relação entre os três atributos numéricos através de um gráfico 3D e de gráficos que traçam a relação par a par entre eles.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data["Credit amount"], data["Duration"], data["Age"])
ax.set_xlabel("Credit amount")
ax.set_ylabel("Duration")
ax.set_zlabel("Age")

In [ ]:
def scatters(data, h=None, pal=None):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8))
    sns.scatterplot(x="Credit amount",y="Duration", hue=h, palette=pal, data=data, ax=ax1)
    sns.scatterplot(x="Age",y="Credit amount", hue=h, palette=pal, data=data, ax=ax2)
    sns.scatterplot(x="Age",y="Duration", hue=h, palette=pal, data=data, ax=ax3)
    plt.tight_layout()

In [ ]:
scatters(data)

Estas análises sugerem a existência de uma correlação positiva entre Credit amount e Duration. Esta correlação é pertinente, visto que valores maiores de créditos tendem a ser pagos em um prazo maior.

Podemos observar, também, a distribuição de cada atributo numérico.

In [ ]:
def distributions(df):
    fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(8,8))
    sns.distplot(df["Age"], ax=ax1)
    sns.distplot(df["Credit amount"], ax=ax2)
    sns.distplot(df["Duration"], ax=ax3)
    plt.tight_layout()

In [ ]:
distributions(data_num)

Com a análise das distribuições, percebemos de forma ainda mais clara que os atributos variam em escalas diferentes. Em algoritmos que lidam com medidas de proximidade, como é o caso do k-means, é importante normalizar os dados para que os valores dos diferentes atributos estejam em ordens de grandeza similares, e assim exerçam o mesmo impacto no aprendizado. Iremos utilizar o método StandardScaler (também chamado por padronização).

In [ ]:
scaler = StandardScaler()
data_num_std = scaler.fit_transform(data_num)
data_num_std = pd.DataFrame(data_num_std, columns=num_columns)

In [ ]:
## Visualizando novamente os dados, após padronização
## (Apenas para entender, visualmente, que o padrão na relação dos dados não muda)
scatters(data_num_std)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_num_std["Credit amount"], data_num_std["Duration"], data_num_std["Age"])
ax.set_xlabel("Credit amount")
ax.set_ylabel("Duration")
ax.set_zlabel("Age")

### **Aplicação do K-means**

Nesta seção, o grupo deve realizar a aplicação do algoritmo [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) no dataframe `data_num_std`, seguindo os passos listados abaixo:


1.   Faça a análise do k-means para diferentes números de clusters. Sugere-se testar de 1 a 20. Crie um vetor para armazenar a soma do quadrado das distâncias das instâncias até o centróide mais próximo durante o loop (também chamado de inércia, e disponível no campo *inertia_* do objeto retornado pelo método KMeans, por exemplo, kmeans.inertia_)  
2.   Faça um gráfico da inércia (eixo y) para os diferentes valores de k (eixo x), a fim de determinar o melhor valor de k pelo método do cotovelo (Elbow method).
3.   Escolha o melhor valor de k para os dados, repetindo a execução do k-means com o k escolhido e gerando a configuração final de clusters.



In [ ]:
max_clusters = 0 #@param {type:"integer"}
inertias=[]
for ii in range(1,max_clusters):
   ## executar o k-means com ii clusters
   ## armazenar o respectivo valor de .inertia_ no vetor inertia

In [ ]:
plt.figure()
plt.plot(range(1,max_clusters),inertias, marker='o')

In [ ]:
num_clusters = 0 #@param {type:"integer"}

kmeans_final = KMeans(num_clusters)
kmeans_final.fit(data_num_std)
labels=kmeans_final.labels_

A célula abaixo gera uma versão do conjunto de dados com uma coluna 'cluster' adicionada aos dados, indicando o índice do cluster ao qual foi designada cada instância. Esta versão será útil para visualização dos dados e para a interpretação dos resultados.

In [ ]:
clusters_config_std = pd.concat([data_num_std, data_cat, pd.DataFrame({'cluster':labels})], axis=1)
clusters_config_std.head()

Também é possível fazer o mesmo para o dataframe original, isto é, sem a normalização por padronização, para eventualmente facilitar a interpretação dos resultados por meio dos gráficos. Apenas é importante lembrar que os agrupamentos são definidos a partir dos dados normalizados.

In [ ]:
clusters_config_original = pd.concat([data_num, data_cat, pd.DataFrame({'cluster':labels})], axis=1)
clusters_config_original.head()

### **Visualização do resultado do agrupamento**

Para visualizar o resultado do agrupamento em um gráfico de 2D, vamos usar a estratégia de *Principal Component Analysis* (PCA), que faz uma projeção dos dados a partir da combinação linear dos atributos (dimensões originais).

Esta transformação será realizada tanto nos dados usados no algoritmo k-means, como nos centróides dos clusters gerados pelo algoritmo.

Cada ponto será representado pelas coordenadas {PC1, PC2} (onde PC = *Principal Component*) e a cor do ponto no gráfico corresponde ao seu respectivo cluster.

In [ ]:
from sklearn.decomposition import PCA
pca_2 = PCA(2)
pca_2_result = pca_2.fit_transform(data_num_std)

## obtém os centrois do k-means e aplica a transformação por PCA
centroids = kmeans_final.cluster_centers_
centroids_pca = pca_2.transform(centroids)

## plota a figura, colorindo os pontos de acordo com o respectivo cluster.
sns.set(style='white', rc={'figure.figsize':(9,6)},font_scale=1.1)

plt.scatter(x=pca_2_result[:, 0], y=pca_2_result[:, 1], c=labels, cmap='viridis')
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1],
            marker='x', s=169, linewidths=3,
            color='black', zorder=10,lw=3)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('Clustered Data (PCA visualization)',fontweight='bold')
plt.show()

### **Interpretação dos resultados**

A análise de agrupamentos tem como resultado a geração de **clusters** que são definidos com base na **alta similaridades entre as características** (ou atributos) **das instâncias**. Assim, usualmente o resultado do agrupamento precisa ser explorado visualmente, através de gráficos, tabelas ou estatística descritiva, e interpretado.

Nesta seção do documento, os grupos deverão implementar suas estratégias para **analisar e comparar** a distribuição dos atributos entre os clusters encontrados. **O objetivo é traçar um perfil dos clientes que se encaixa em cada cluster obtido, com base nos atributos disponíveis** (numéricos e categóricos, incluindo aqueles que não foram usados para realizar o agrupamento). Ao final da análise, os grupos devem ser capazes de realizar uma descrição de cada cluster em termos do perfil de cliente que ele melhor representa.

Por exemplo... Algum cluster está associado com pessoas mais jovens? Como está distribuído o valor de crédito em cada cluster? Existe uma proporção maior de mulheres em algum cluster? Existe alguma relação entre os clusters e o tipo de emprego, de moradia, ou de contas (poupança e correnta)? etc. Os grupos podem (e devem) usar a criatividade para realizar a análise dos dados e a interpretação dos dados.

Sugere-se uso de recursos como média e desvio padrão, gráfico de barras, graficos de boxplot, histogramas, ou outros apropriados para análise de distribuição (de acordo com o tipo de atributo, numérico ou categórico).

Os resultados devem ser apresentados e discutidos no próprio notebook do Google Colab ou, alternativamente, em relatório em PDF a ser entregue junto com o notebook.




### **Exemplos de gráficos**

Os exemplos abaixos são dados sem considerar a informação de clusters, apenas os atributos originais, para auxiliar os alunos em algumas visualizações possíveis. Outras visualizações podem ser utilizadas a critério dos alunos.

#### Analisando a distribuição de uma variável numérica para subgrupos (dados por variável categórica)

In [ ]:
# boxplots
for coluna in data_num.columns:  ## usando apenas colunas numéricas com base no dataframe data_num
    sns.boxplot(x='Sex', y=coluna, data=clusters_config_original)
    plt.title(f'Distribuição do atributo {coluna} por Sexo')
    plt.show()

#### Analisando a distribuição de uma variável categórica para subgrupos (dados por variável categórica)

In [ ]:
# barplots
for coluna in ['Job','Housing']:  ## usando apenas colunas numéricas com base no dataframe data_num
    sns.countplot(x=coluna, data=clusters_config_original, hue='Sex')
    plt.title(f'Distribuição do atributo {coluna} por Sexo')
    plt.show()